In [59]:
import pandas as pd
import numpy as np

#스케일링
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# 차원축소
from sklearn.decomposition import PCA

# 샘플링
from sklearn.model_selection import train_test_split

# 모델링
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

#평가
# mas, mse, rmse, mape, 결정계수
from sklearn.metrics import mean_squared_error

#그리드서치
from sklearn.model_selection import GridSearchCV

#######################
####데이터 읽어오기####
#######################
train_X = pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainX.csv",encoding='euc-kr')
train_y= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_trainy.csv",encoding='euc-kr')
test_X= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/Jeju_testX.csv",encoding='euc-kr')
sub= pd.read_csv("https://raw.githubusercontent.com/Datamanim/jeju/main/subExample.csv",encoding='euc-kr')
#'시도명', '읍면동명'


#print(train_X.head(3))
#print(train_y.head(3))
#print(test_X.head(3))
#print(sub.head(3))
#print(train_X.info())
#print(train_y.info())
#print(test_X.info())
#print(train_X.describe())

#print(len(train_X))
#결측치처리, 이상치처리, 파생변수생성, 스케일러만들고fit
total=pd.concat([train_X, test_X], axis=0)
#print(total.iloc[:22448]) # 경계는 22448
total['일자']=pd.to_datetime(total['일자'])
total['year']=total['일자'].dt.year
total['month']=total['일자'].dt.month
total['day']=total['일자'].dt.day
total.drop('일자', axis=1, inplace=True)
total['거주방문비율']=total['거주인구']/total['방문인구']
total['거주근무비율']=total['거주인구']/total['근무인구']
total['근무방문유동비율']=total['총 유동인구']/(total['근무인구']+total['방문인구'])
total['속도소요시간']=total['평균 속도']*total['평균 소요 시간']
total=pd.get_dummies(total, columns=['시도명', '읍면동명'])
rscaler=RobustScaler()
rscaler.fit(total)

'''
['거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도',
       '평균 소요 시간', '평균 기온', '일강수량', '평균 풍속', 'year', 'month', 'day', '거주방문비율',
       '거주근무비율', '근무방문유동비율', '속도소요시간']
['시도명', '읍면동명'] 
'''



def preprocessing(data):
    data=pd.DataFrame(rscaler.transform(data), columns=data.columns)  ## 데이터프레임화
    return data
X_train=preprocessing(total.iloc[:22448])

xy=pd.concat([X_train, train_y], axis=1)
xy=xy.drop('id', axis=1)
corr=xy.corr()['교통량']
#print(corr[corr>0.1])
corr_del_index=corr[corr<0.08].index
X_train.drop(corr_del_index, axis=1, inplace=True)

pca=PCA()
pca.fit(X_train)
print(pca.explained_variance_ratio_.cumsum())
pca=PCA(n_components=10)
pca.fit(X_train)

def pca_data(data):
    data=pd.DataFrame(pca.transform(data))   ## 데이터프레임화
    return data
X_train=pca_data(X_train)

train_x, val_x, train_y, val_y = train_test_split(X_train, train_y['교통량'], test_size=0.2, random_state=42)
#print(len(train_x))
#print(len(val_x))
#print(len(train_y))
#print(len(val_y))

'''
lm=SGDRegressor()
rf=RandomForestRegressor()
gb=GradientBoostingRegressor()
xgb=XGBRegressor()

lm.fit(train_x, train_y)
rf.fit(train_x, train_y)
gb.fit(train_x, train_y)
xgb.fit(train_x, train_y)

print(lm.score(train_x, train_y))
print(lm.score(val_x, val_y))
print(mean_squared_error(val_y, lm.predict(val_x)))
print("=====")
print(rf.score(train_x, train_y))
print(rf.score(val_x, val_y))
print(mean_squared_error(val_y, rf.predict(val_x)))
print("=====")
print(gb.score(train_x, train_y))
print(gb.score(val_x, val_y))
print(mean_squared_error(val_y, gb.predict(val_x)))
print("=====")
print(xgb.score(train_x, train_y))
print(xgb.score(val_x, val_y))
print(mean_squared_error(val_y, xgb.predict(val_x)))
print("=====")
'''
rf=RandomForestRegressor()
rf.fit(train_x, train_y)
print(rf.score(train_x, train_y))
print(rf.score(val_x, val_y))
print(mean_squared_error(val_y, rf.predict(val_x)))
print("=====")

param_grid=[{
#    {'bootstrap': [True, False],
 'max_depth': [30,60,90],
# 'max_features': ['auto', 'sqrt'],
# 'min_samples_leaf': [1, 2, 4],
# 'min_samples_split': [2, 5, 10],
 'n_estimators': [30,50,70],}
]
#grid=GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, n_jobs=-1)
#grid=RandomForestRegressor(n_estimators=30, max_depth=30, random_state=42)
#grid.fit(train_x, train_y)
#print(grid.best_params_)
#print(grid.score(train_x, train_y))
#print(grid.score(val_x, val_y))
#print(mean_squared_error(val_y, grid.predict(val_x)))

[0.71293538 0.8678854  0.91063184 0.9403201  0.9633001  0.97062257
 0.97420574 0.97776246 0.98131864 0.98487392 0.98841442 0.99147203
 0.99441087 0.99681032 0.99886592 0.99981987 1.         1.        ]
0.9954043475839945
0.9701509018634907
9127.91096744233
=====
